# 背景减除

In [3]:
import cv2 as cv
import numpy as np

filename = 'data/vtest.avi'
cap = cv.VideoCapture(filename)              # 0-摄像头输入/filename
cv.namedWindow('image', cv.WINDOW_NORMAL)    # 设置显示窗口

# 得到视频参数
width, height = cap.get(3), cap.get(4)       # 视频帧宽高
fps = cap.get(5)                             # 视频帧率
count = cap.get(7)                           # 视频帧数
duration = count / fps
print(('width: {}, height: {}'.format(width, height)))
print(('duration: %.1f s, count: %s, fps: %s' % (duration, count, fps)))
# print('fourcc: ', chr(int(fourcc)))

# fgbg = cv.createBackgroundSubtractorMOG2()  # 生成前景掩码的方法:MOG2/KNN   
fgbg = cv.createBackgroundSubtractorKNN()     # 默认检测影子，detectShadows=True

while cap.isOpened():               # 是否初始化摄像头
    ret, frame = cap.read()         # 得到的frame可以当作图片来处理

    # 视频左上角显示当前帧编号
    cv.rectangle(frame, (10, 2), (50, 20), (255, 255, 255), -1)
    cv.putText(frame, str(int(cap.get(cv.CAP_PROP_POS_FRAMES))),(15, 15), cv.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,0),1, cv.LINE_AA)

    if not ret:
        break
    # cv.imshow('frame', frame)     # 视频帧显示

    # 前景掩模和背景提取
    fgmask = fgbg.apply(frame)
#     _, fgmask = cv.threshold(
#         fgmask, 200, 255, cv.THRESH_BINARY)   # 阈值分割,去除脚下的影子

    # 形态学滤波:去除噪点
#     Element = cv.getStructuringElement(cv.MORPH_CROSS, (5, 5))
#     Open = cv.morphologyEx(fgmask, cv.MORPH_OPEN, Element, 2)
#     fgmask = cv.morphologyEx(Open, cv.MORPH_CLOSE, Element, 2)

    # cv.imshow('fgmask', fgmask)       # 前景掩模
    bgImage = fgbg.getBackgroundImage()
    # cv.imshow('background', bgImage)  # 背景视频

    fgmask = cv.cvtColor(fgmask, cv.COLOR_GRAY2BGR)
    dst = np.hstack((frame, fgmask))   # frame, fgmask, bgImage
    # print(frame.shape, fgmask.shape, bgImage.shape)
    cv.imshow('image', dst)

    key = cv.waitKey(int(1000 / fps))      # 30 设置视频的播放速度:与原视频相同
    if key == 27:                          # ESC的ASCII码为27
        break
    elif key == ord('s'):        # 截取帧并保存
        cv.imwrite('frame.png', frame)
        cv.imwrite('fgmask.png', fgmask)

cap.release()
cv.destroyAllWindows()

width: 768.0, height: 576.0
duration: 79.5 s, count: 795.0, fps: 10.0


# Meanshift和Camshift
视频对象跟踪
- cv.meanShift()
- cv.CamShift()

## Meanshift

In [1]:
import numpy as np
import cv2 as cv

cap = cv.VideoCapture('data/slow_traffic_small.mp4')

ret, frame = cap.read()         # 获取视频的第一帧
# 设置ROI b-box的初始位置
x, y, w, h = 300, 200, 100, 50  # simply hardcoded the values
track_window = (x, y, w, h)

# 得到ROI图像
roi = frame[y:y + h, x:x + w]
hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)

# 得到ROI图像的归一化直方图
mask = cv.inRange(hsv_roi, np.array((0., 60., 32.)),
                  np.array((180., 255., 255.)))
roi_hist = cv.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv.normalize(roi_hist, roi_hist, 0, 255, cv.NORM_MINMAX)

# 设置迭代终止条件，可以是10次迭代，有可以至少移动1个像素
term_crit = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)

while (1):
    ret, frame = cap.read()
    
    if ret == True:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)      # 直方图反投影
        ret, track_window = cv.meanShift(dst, track_window, term_crit)   # 应用meanshift来获取track_window的新位置
        # dst—ROI的直方图反投影图  track_window—ROI b-box坐标  term_crit—迭代终止条件
        
        # 在图像上绘制ROI b-box
        x, y, w, h = track_window
        img2 = cv.rectangle(frame, (x, y), (x + w, y + h), 255, 2)
        cv.imshow('img2', img2)
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break

cap.release()
cv.destroyAllWindows()

## Camshift
可以自适应地更新跟踪窗口的大小

In [2]:
import numpy as np
import cv2 as cv
import argparse

cap = cv.VideoCapture('data/slow_traffic_small.mp4')

ret, frame = cap.read()         # 获取视频的第一帧

# 设置ROI b-box的初始位置
x, y, w, h = 300, 200, 100, 50  # simply hardcoded the values
track_window = (x, y, w, h)

# 得到ROI图像
roi = frame[y:y+h, x:x+w]
hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)

# 得到ROI图像的归一化直方图
mask = cv.inRange(hsv_roi, np.array((0., 60., 32.)),
                  np.array((180., 255., 255.)))
roi_hist = cv.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv.normalize(roi_hist, roi_hist, 0, 255, cv.NORM_MINMAX)

# 设置迭代终止条件，可以是10次迭代，有可以至少移动1个像素
term_crit = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)

while(1):
    ret, frame = cap.read()
    if ret == True:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)     # 直方图反投影
        ret, track_window = cv.CamShift(dst, track_window, term_crit)   # 应用meanshift来获取track_window的新位置
        # dst—ROI的直方图反投影图  track_window—ROI b-box坐标  term_crit—迭代终止条件
        # 返回ret包含ROI的坐标点信息
        
        # 在图像上绘制ROI b-box
        pts = cv.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv.polylines(frame, [pts], True, 255, 2)
        cv.imshow('img2', img2)
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break

cap.release()
cv.destroyAllWindows()

# 光流
## OpenCV中的Lucas-Kanade
cv.calcOpticalFlowPyrLK()

In [1]:
import numpy as np
import cv2 as cv
import argparse

cap = cv.VideoCapture('data/slow_traffic_small.mp4')
# 用于Shi-Tomasi角点检测的参数
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# lucas kanade光流参数
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

color = np.random.randint(0,255,(100,3))    # 为检测到的角点创建随机颜色
ret, old_frame = cap.read()                 # 获取视频的第一帧
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)   # 第1帧进行Shi-Tomasi角点检测

mask = np.zeros_like(old_frame)    # mask:用于绘制关键点连线
while(1):
    ret,frame = cap.read()
    if ret == True:
        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        # 利用相邻两帧计算光流
        p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        '''Lucas Kanade光流法参数
        p1: p0在当前帧进行光流检测后的位置
        st: 是否是运动的角点
        err: 是否出错
        old_gray: 之前进行角点检测的帧
        frame_gray: 当前帧
        p0: 之前帧检测到的角点'''
        
        # 选择良好点:运动角点
        good_new = p1[st==1]
        good_old = p0[st==1]
        # 绘制跟踪点
        for i,(new,old) in enumerate(zip(good_new, good_old)):
            a,b = new.ravel()
            c,d = old.ravel()
            mask = cv.line(mask, (a,b),(c,d), color[i].tolist(), 2)
            frame = cv.circle(frame,(a,b),5,color[i].tolist(),-1)
        img = cv.add(frame,mask)
        cv.imshow('frame',img)
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
        # 现在更新之前的帧和点
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1,1,2)
    else:
        break
    
cap.release()
cv.destroyAllWindows()

## OpenCV中的密集光流
cv.calcOpticalFlowFarneback()

In [2]:
import numpy as np
import cv2 as cv
cap = cv.VideoCapture('data/vtest.avi')
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255  

while (1):
    ret, frame2 = cap.read()
    if ret == True:
        next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
        flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        '''
        前两个参数与上述参数一样，第三个参数跟返回值一样
        prvs, next     上一帧和当前帧图像(单通道)
        pyr_scale=0.5  构建图像金字塔尺度
        levels=3       图像金字塔层数
        winsize=15     窗口尺寸,值越大探测高速运动的物体越容易，但是越模糊，同时对噪声的容错性越强
        iterations=3   对每层金字塔的迭代次数
        poly_n=5       每个像素中找到多项式展开的邻域像素的大小。越大越光滑，也越稳定
        poly_sigma=1.2 高斯标准差，用来平滑倒数
        flags=0        光流的方式，有OPTFLOW_USE_INITIAL_FLOW 和OPTFLOW_FARNEBACK_GAUSSIAN 两种
        flow           返回关键点的坐标
        '''
        mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
        # 笛卡尔坐标->极坐标
        # mag-极径 ang-幅角(rad)  x坐标list, y坐标list
        hsv[..., 0] = ang * 180 / np.pi / 2                            # rad->°
        hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)  # 数值归一化
        bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
        cv.imshow('frame2', bgr)
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
        elif k == ord('s'):
            cv.imwrite('opticalfb.png', frame2)
            cv.imwrite('opticalhsv.png', bgr)
        prvs = next
    else:
        break

cap.release()
cv.destroyAllWindows()